In [187]:
import os
import json
import openai
import tiktoken
import numpy as np
from dotenv import find_dotenv, load_dotenv

In [5]:
load_dotenv(find_dotenv())

True

In [112]:
model_id = "gpt-3.5-turbo-1106"
client = openai.OpenAI(api_key=os.environ["OPENAI_API_KEY"])
tokenizer = tiktoken.encoding_for_model(model_id)

In [ ]:
classifier_prompt_1 = """
<s>[INST]
# TASK DESCRIPTION
You are to analyze user queries regarding industrial robotic operations and translate these into specific function calls. Your role is to bridge human instructions and the technical execution of a robot.
You will extract the information and values from the USER QUERY and form a JSON output which will represent a fuction call, you will fill in the JSON 
with the necessary information. 

# JSON PARAMETER VALUES
- functions: This is an array that represents all the function calls needed to perform the actions requested buy the USER QUERY. The structure of each function is defined below. These
functions are separated in the USER QUERY by words like "and", "then" ... you need to fill the array using all the actions defined by the USER QUERY while respectiong all the rules defined below.
- function_name: Which function does the robot need to perform. Can be only one of these: move_tcp, move_joint, get_joint_values
- input_name: Input parameter name for the function. Every function has a defined set of input parameters, what they mean, how they are named and what type of value they can be.
- input_value: The value of the previously mentioned function input parameter. It's type is defined within the function explaination
- inputs: An array of input_name, input_value pairs. One inputs array defines all the input parameters for a function call. This array must be complete according to the definition of the chosen function_name 

# FUNCTION MEANING
- move_tcp: Moves the robot's tool center point (TCP). This function takes as inputs: x, y, z, q1, q2, q3, q4
x, y ,z can only have number values, they represent the desired position of the TCP for each axis and are expressed in milimeters, this unit is implicit. Example: "Move TCP to the right by 0.1m" results in input_name: x and input_value: 10 
q1, q2, q3, q4 can only have number values and represent the quaternion value of the TCP orientation, defalut should be the TCP pointing down with it's z axis 
- move_joint: Rotates or moves a specific robot joint. This funciton takes as inputs joint, angle 
joint is the index of the joint that needs to rotate. Joints are indext 0, 1, 2 ... The joint 0 is also often reffered to as "base joint", "robot base" or just "base"
angle is the amount the specified joint needs to rotate and is expressed in radians
Example: "Rotate the third joint by 90 degrees." results in joint: 2 angle: 1.5708
- get_joint_values: Retrieves current status of the robot's joints. This function takes no input parameters! Example: "What is the position of the fifth joint?"

# RESPONSE FORMAT
- Include only the necessary functions directly implied by the query.
- Maintain the order of functions as implied by the sequence of actions in the query.
- In case of ambiguity, provide the most likely function while noting the uncertainty.

# ADDITIONAL GUIDANCE
- Focus on the verbs and technical terms in the query to determine the appropriate function.
- If a query involves actions not covered by the functions, such as maintenance requests, indicate that the query falls outside the function list.
- Consider the practical aspects of robotic operations when interpreting instructions.
- Treat "base" as equivalent to the first robot "joint".

USER QUERY: {user_query}
[/INST]
"""

In [116]:
prompt = """
# TASK DESCRIPTION
You are to generate dataset for LLM fine-tuning that will have pairs of user queries regarding industrial robotic operations and those queries trnaslated into specific function calls.
Your role is to bridge human instructions and the technical execution of a robot.
You will generate USER QUERY and appropriate JSON output which will represent a fuction call, you will fill in the JSON 
with the necessary information. 

# JSON PARAMETER VALUES
- query: Generated user query.
- functions: This is an array that represents all the function calls needed to perform the actions requested buy the USER QUERY. The structure of each function is defined below. These
functions are separated in the USER QUERY by words like "and", "then", ",", ".", ";" ... you need to fill the array using all the actions defined by the USER QUERY while respectiong all the rules defined below.
- function_name: Which function does the robot need to perform. Can be only one of these: move_tcp, move_joint, get_joint_values.
- input_name: Input parameter name for the function. Every function has a defined set of input parameters, what they mean, how they are named and what type of value they can be.
- input_value: The value of the previously mentioned function input parameter. It's type is defined within the function explaination.

# FUNCTIONS AND MEANINGS
- move_tcp: Moves the robot's tool center point (TCP). This function takes as inputs: x, y, z, q1, q2, q3, q4
x, y ,z can only have number values and units, they represent the desired position of the TCP for each axis and units in wich are expressed. EXAMPLE: "Move TCP along x axis for 0.1m" results in input_name: x and input_value: 0.1, unit: m
q1, q2, q3, q4 can only have number values and represent the quaternion value of the TCP orientation, defalut should be the TCP pointing down with it's z axis.
- move_joint: Rotates or moves a specific robot joint. This funciton takes as inputs joint, angle 
joint is the index of the joint that needs to rotate. Joints are indext 0, 1, 2 ... The joint 0 is also often reffered to as "base joint", "robot base" or just "base".
Angle is the amount the specified joint needs to rotate. You need to detect unit for angle that user what from the query.
Example: "Rotate the third joint by 90 degrees." results in joint: 2 angle: 90, unit: deg
- get_joint_values: Retrieves current status of the robot's joints. This function takes no input parameters! Example: "What is the position of the fifth joint?"

# RESPONSE FORMAT
- In move_tcp request, user query MUST contain value for how much robot arm needsto move.
- For inputs only return unit if it is needed, if value doesn't require units do not return them.
- Return only values specified in query, DO NOT return default values.
- Include only the necessary functions directly implied by the query.
- Maintain the order of functions as implied by the sequence of actions in the query.

# GUIDANCE
- Focus on the verbs and technical terms in the query to determine the appropriate function.
- Query needs to involves specified movement or rotation like: 'TCP' or 'joint' and appropriate arguments.
- In cases of compound queries (multiple actions), break down each action to decide the appropriate function.
- Consider the practical aspects of robotic operations when interpreting instructions.
- Treat "base" as equivalent to the first robot "joint".
- Determenistic queries: Queries need to contain all information needed for function to be called. Avoid ambiguos queries. 
- Be precise: Ensure the functions match the specific actions requested in the query.
- Human like queries: Generate queries that have additional context but do not infulence desired outcome. 
- Query can contain 1, 2, 3 or more actions.

# RESPONSE EXAMPLE:
{
    "query" : <generated_query>,
    "functions" : [
        {
            "function_name" : <name_from_functions_and_meanings>
            "inputs" : [
                {
                    "name" : <name>,
                    "value" : <value>,
                    "unit" : <unit>,
                },
                {
                    "name" : <name>,
                    "value" : <value>,
                    "unit" : <unit>,
                },
            ]
        },
        {
            "function_name" : <name_from_functions_and_meanings>
            "inputs" : [
                {
                    "name" : <name>,
                    "value" : <value>,
                    "unit" : <unit>,
                }
            ]
        }
    ]

}

"""

In [117]:
prompt_tokenized = tokenizer.encode(prompt)
len(prompt_tokenized)

995

In [196]:
rand_num = np.random.rand().__round__(5)
rand_num

0.51507

In [228]:
prompt_updated = """
# TASK DESCRIPTION {rand_num}
Create a dataset for LLM fine-tuning consisting of user queries about industrial robotic operations and their corresponding JSON function calls. Focus on generating queries with varying complexity levels, from simple to advanced, and provide examples in real-world scenarios.

# JSON PARAMETER VALUES
- query: User-generated query, varying in complexity (contains different number of functions in query).
- functions: Array of function calls derived from the query, identified using separators like "and", "then", ",", etc.
- function_name: Specific robot function ("move_tcp", "move_joint", "get_joint_values").
- input_name: Name of the function's input parameter, specifying the type (integer, float, string).
- input_value: Value of the input parameter, adhering to the specified type.

# FUNCTIONS EXPLAINED WITH EXAMPLES
- move_tcp: Moves the robot's TCP. 
    Inputs: x, y, z (position; type: float), q1, q2, q3, q4 (quaternion values; type: float), unit (correlates to x, y, z; type: str, ["m", "cm", "mm"]). 
    Example: "Move TCP to coordinates (0.5, 0.3, 0.7) m" translates to x: 0.5, y: 0.3, z: 0.7; unit: m
- move_joint: Rotates/moves a robot joint. 
    Inputs: joint (index number; type: array of integers; note: joints go from 0 to n-1), angle (degrees or radians, type: array of floats), unit (correlates to angle; type: str, ["rad", "deg"]). 
    Example: "Rotate joint 2 by 45 degrees" results in joint: [2], angle: [45], unit: deg.
- get_joint_values: Retrieves robot joint statuses, no input parameters required. 
    Example: "Get the current status of robot joints."

# GUIDANCE
- Focus on verbs and technical terms for function selection.
- Ensure queries are deterministic and precise.
- Queries should vary in complexity, from direct instructions to those requiring contextual understanding.
- Real-World Scenarios: Frame queries in practical industrial settings.

# RESPONSE FORMAT
- Include only necessary functions as per the query.
- Maintain the sequence of functions as in the query.
- Specify units only when required, omit default values.

# JSON FORMAT
{{
    "query": "<generated_query>",
    "functions": [
        {{
            "function_name": "<name>",
            "inputs": [
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}},
                {{"name": "<name>", "value": "<value>", "unit": "<unit>"}}
            ]
        }},
        {{
            "function_name": "<name>",
            "inputs": [{{"name": "<name>", "value": "<value>", "unit": "<unit>"}}]
        }}
    ]
}}

# GENERATED EXAMPLES:
    NOTE: DO NOT copy these examples, only use them as a reference!
    {{
        'query': 'Move robot tcp along x and y for 1000mm',
        'functions': [{{'function_name': 'move_tcp',
            'inputs': [
                {{'name': 'x', 'value': 1000.0, 'unit': 'mm'}},
                {{'name': 'y', 'value': 1000.0, 'unit': 'mm'}}
                ]
            }}
        ]
    }}
    {{
        'query': 'Move robot sixth joint for -30 degrees',
        'functions': [{{'function_name': 'move_joint',
            'inputs': [
                {{'name': 'joint', 'value': [5], 'unit': None}},
                {{'name': 'angle', 'value': [-30.0], 'unit': 'deg'}}
                ]
            }}
        ]
    }}
    {{
        'query': 'Give me robot joints info',
        'functions': [{{'function_name': 'get_joint_values',
            'inputs': []
            }}
        ]
    }}
    {{
        'query': 'Rotate robot base for 45 and move TCP along x axis for 50 milimeters.',
        'functions': [
            {{'function_name': 'move_joint',
                'inputs': [
                    {{'name': 'joint', 'value': [0], 'unit': None}},
                    {{'name': 'angle', 'value': [45.0], 'unit': 'deg'}}
                ]
            }},
            {{'function_name': 'move_tcp',
                'inputs': [{{'name': 'x', 'value': 50.0, 'unit': 'mm'}}]
            }}
        ]
    }}
    {{
        'query': 'I want you to rotate joint 2 for 30 and joint 7 for 45 degrees then joint 3 for pi/4',
        'functions': [
            {{'function_name': 'move_joint',
                'inputs': [
                    {{'name': 'joint', 'value': [2, 7], 'unit': None}},
                    {{'name': 'angle', 'value': [30.0, 45.0], 'unit': 'deg'}}
                    ]
            }},
            {{'function_name': 'move_joint',
                'inputs': [
                    {{'name': 'joint', 'value': [3], 'unit': None}},
                    {{'name': 'angle', 'value': [0.785398], 'unit': 'rad'}}
                    ]
            }}
        ]
    }}
"""

In [229]:
prompt_tokenized = tokenizer.encode(prompt_updated)
len(prompt_tokenized)

1177

In [242]:
response = client.chat.completions.create(
    model=model_id,
    temperature=1,
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "system",
            "content": "You are a helpful assistant designed to output JSON.",
        },
        {
            "role": "user",
            "content": prompt_updated.format(rand_num=np.random.rand().__round__(5)),
        },
    ],
)

Witch system prompt

In [243]:
gen_data = json.loads(response.choices[0].message.content)
gen_data

{'query': 'Move robot tcp along z for 300 milimeters and then get the current status of robot joints',
 'functions': [{'function_name': 'move_tcp',
   'inputs': [{'name': 'z', 'value': 300.0, 'unit': 'mm'}]},
  {'function_name': 'get_joint_values', 'inputs': []}]}

Without system prompt

In [248]:
response = client.chat.completions.create(
    model=model_id,
    temperature=1,
    response_format={"type": "json_object"},
    messages=[
        {
            "role": "user",
            "content": prompt_updated.format(rand_num=np.random.rand().__round__(5)),
        },
    ],
)

In [249]:
gen_data = json.loads(response.choices[0].message.content)
gen_data

{'query': 'Move robot tcp to position (0.6, 0.8, 0.9) meters',
 'functions': [{'function_name': 'move_tcp',
   'inputs': [{'name': 'x', 'value': 0.6, 'unit': 'm'},
    {'name': 'y', 'value': 0.8, 'unit': 'm'},
    {'name': 'z', 'value': 0.9, 'unit': 'm'}]}]}